In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,29250
2,Huelva,Confirmados PDIA 14 días,1030
3,Huelva,Tasa PDIA 14 días,"200,7132139446967"
4,Huelva,Confirmados PDIA 7 días,542
5,Huelva,Total Confirmados,29426
6,Huelva,Curados,25910
7,Huelva,Fallecidos,370


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  29250.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7635.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 817 personas en los últimos 7 días 

Un positivo PCR cada 470 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,29250.0,542.0,1030.0,513170.0,105.618021,200.713214,132.0
Huelva-Costa,17015.0,281.0,488.0,289548.0,97.047812,168.538550,70.0
Condado-Campiña,9304.0,228.0,473.0,156231.0,145.937746,302.756815,57.0
Huelva (capital),7635.0,176.0,306.0,143837.0,122.360728,212.740811,41.0
Almonte,1629.0,79.0,156.0,24507.0,322.356878,636.552822,25.0
Moguer,1131.0,23.0,52.0,21867.0,105.181323,237.801253,10.0
Isla Cristina,2098.0,34.0,45.0,21393.0,158.930491,210.349180,9.0
Bollullos Par del Condado,876.0,42.0,89.0,14387.0,291.930215,618.614027,8.0
Punta Umbría,736.0,22.0,29.0,15355.0,143.275806,188.863562,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cortelazor,8.0,1.0,3.0,299.0,334.448161,1003.344482,0.0
Galaroza,37.0,7.0,10.0,1382.0,506.512301,723.589001,2.0
Almonte,1629.0,79.0,156.0,24507.0,322.356878,636.552822,25.0
Bollullos Par del Condado,876.0,42.0,89.0,14387.0,291.930215,618.614027,8.0
Lucena del Puerto,234.0,1.0,16.0,3261.0,30.665440,490.647041,NaN
Rociana del Condado,529.0,13.0,28.0,7939.0,163.748583,352.689256,2.0
Condado-Campiña,9304.0,228.0,473.0,156231.0,145.937746,302.756815,57.0
Escacena del Campo,149.0,3.0,6.0,2287.0,131.176213,262.352427,1.0
Moguer,1131.0,23.0,52.0,21867.0,105.181323,237.801253,10.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Lucena del Puerto,234.0,1.0,16.0,3261.0,30.665440,490.647041,0.0,0.062500
Bonares,259.0,4.0,13.0,6060.0,66.006601,214.521452,0.0,0.307692
Cortelazor,8.0,1.0,3.0,299.0,334.448161,1003.344482,0.0,0.333333
Rosal de la Frontera,29.0,1.0,3.0,1697.0,58.927519,176.782557,0.0,0.333333
Ayamonte,1296.0,9.0,25.0,21104.0,42.645944,118.460955,0.0,0.360000
Beas,202.0,3.0,8.0,4341.0,69.108500,184.289334,0.0,0.375000
Aljaraque,1016.0,10.0,24.0,21474.0,46.567943,111.763062,4.0,0.416667
Moguer,1131.0,23.0,52.0,21867.0,105.181323,237.801253,10.0,0.442308
Cartaya,1291.0,16.0,36.0,20083.0,79.669372,179.256087,2.0,0.444444
